In [39]:
import os
import pandas as pd
import zipfile
import openpyxl
from win32com.client import Dispatch
import time

In [40]:
pasta_de_arquivos = r'seu/diretorio_dos_arquivos'

pasta_mes = pasta_de_arquivos
pasta_principal = os.path.dirname(os.path.dirname(pasta_mes))




In [41]:
lista_todos_clientes = []

def compacta_pastas(caminho_principal, mes):
    # Itera sobre cada pasta dentro do caminho principal
    for cliente in os.listdir(caminho_principal):
        caminho_cliente = os.path.join(caminho_principal, cliente)
        lista_todos_clientes.append(cliente)

        # Verifica se é uma pasta
        if os.path.isdir(caminho_cliente):
            # Nome do arquivo zip
            nome_zip = f'{mes} 25 - {cliente}.zip'
            caminho_zip = os.path.join(caminho_cliente, nome_zip)
            
            # Cria o arquivo zip
            with zipfile.ZipFile(caminho_zip, 'w') as zipf:
                # Percorre recursivamente todos os arquivos e subpastas
                for root, _, files in os.walk(caminho_cliente):
                    for file in files:
                        arquivo_path = os.path.join(root, file)
                        # Ignora o próprio arquivo zipado recém-criado
                        if arquivo_path == caminho_zip:
                            continue
                        # Adiciona ao zip mantendo a estrutura relativa
                        zipf.write(arquivo_path, os.path.relpath(arquivo_path, caminho_cliente))
            
            print(f"Arquivos de {cliente} compactados em: {caminho_zip}")


In [ ]:
compacta_pastas(pasta_mes, 'JAN')

In [ ]:
caminho_lista_emails = os.path.join(pasta_principal,'arquivo_lista_emails.xlsx')

df = pd.read_excel(caminho_lista_emails)

df

In [ ]:
novos_clientes = [c for c in lista_todos_clientes if c not in df['CLIENTE'].values]

novos_clientes

In [ ]:
df = pd.concat([df, pd.DataFrame({"CLIENTE": novos_clientes})], ignore_index=True)

df

In [46]:
df.to_excel(caminho_lista_emails, index=False)

In [ ]:
df = pd.read_excel(caminho_lista_emails)

df

In [ ]:
import os
import logging
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from email.mime.application import MIMEApplication
#from dotenv import load_dotenv

# Carrega variáveis de ambiente
# load_dotenv()


def send_email(sender_email, to_email, subject, zip_path, cliente):

    # Monta o e-mail
    msg = MIMEMultipart()
    msg['From'] = sender_email
    msg['To'] = ", ".join(to_email)
    msg['Cc'] = 'emails_em_copia'
    msg['Subject'] = subject

    body = """

Mensagem a ser enviada no corpo do email

"""

    # Corpo do e-mail
    msg.attach(MIMEText(body, 'html'))

    # Anexo ZIP
    with open(zip_path, 'rb') as f:
        part = MIMEApplication(f.read(), Name=os.path.basename(zip_path))
        part['Content-Disposition'] = f'attachment; filename="{os.path.basename(zip_path)}"'
        msg.attach(part)

    try:
          server.sendmail(sender_email, to_email, msg.as_string())
          print(f'E-mail enviado para {cliente} com email: {to_email}')

    except Exception as e:
        print(f'Falha no envio para {to_email}: {e}')


In [ ]:
smtp_server = "smtp.office365.com"
smtp_port = 587
sender_email = 'seu_email'
sender_password = 'sua_senha_de_aplicativo'

df_emails_enviados = pd.DataFrame(columns=['CLIENTE'])

with smtplib.SMTP(smtp_server, smtp_port) as server:
    server.starttls()
    server.login(sender_email, sender_password)
    
    for _, row in df.iterrows():  # Itera sobre cada linha
        cliente = row['CLIENTE']  # Ajuste conforme o nome da coluna
        email_cliente = [e.strip() for e in row['EMAIL'].split(';')]  # Ajuste conforme o nome da coluna

        caminho_cliente = os.path.join(pasta_mes, cliente)
        
        # Envia o arquivo compactado para o cliente
        nome_zip = f'KIT JAN 25 - {cliente}.zip'
        caminho_zip = os.path.join(caminho_cliente, nome_zip)
        
        if os.path.exists(caminho_zip):
            assunto = f'KIT - PRESTAÇÃO DE CONTAS | {cliente} | JANEIRO 25'
            send_email(sender_email, email_cliente, assunto, caminho_zip, cliente)

            df_emails_enviados.loc[len(df_emails_enviados)] = {'CLIENTE': cliente}
            
        else:
            print(f'O arquivo {nome_zip} não foi encontrado para {cliente}.')

In [38]:
df_emails_enviados.to_excel(os.path.join(pasta_principal,'emails_enviados.xlsx'))